<a href="https://colab.research.google.com/github/AgoMilan/Ai/blob/main/01_hledej_sperk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -Uqq fastai duckduckgo_search
!pip install fastbook
!pip install fastai

Třída k roztřídění šperků do jednotlivých kategorií

In [ ]:
from fastai.vision.all import *
from pathlib import Path
import shutil
from scipy.spatial.distance import cosine
import os
from PIL import Image

##Třída předpovídá do které kategorie patří testovaný obrázek
class ImageClassifier:
    def __init__(self, model_dir, classified_dir, class_mapping):
        self.model_dir = Path(model_dir)
        self.classified_dir = Path(classified_dir)
        self.class_mapping = class_mapping
        self.learn_predict = load_learner(self.model_dir)

    def classify_and_copy(self, image_path):
        img = PILImage.create(image_path)
        pred, label, probs = self.learn_predict.predict(img)

        label_str = self.class_mapping[str(label)]
        target_dir = self.classified_dir / label_str
        target_path = target_dir / image_path.name

        if target_path.exists():
            print(f"Soubor '{image_path.name}' již existuje v cílovém adresáři '{target_dir}'. Přeskočeno.")
            return

        #shutil.copy(image_path, target_path) # Odkomentovat pokud chci uložit testovaný obrázek do klasifikované kategorie

        print(f"Obrázek '{image_path.name}' klasifikován jako '{label_str}' a zkopírován do '{target_dir}'")
        print("Porovnat v kategorii - ", label_str)


    def process_images(self, unclassified_dir):
        for label_dir in unclassified_dir.glob("*"):
            if label_dir.is_dir():
                for image_path in get_image_files(label_dir):
                    self.classify_and_copy(image_path)



# Definice cest k adresářům
unclassified_dir = Path("/content/drive/MyDrive/KatalogAA")
classified_dir = Path("/content/drive/MyDrive/Katalog/sperk")
model_dir = Path("/content/drive/MyDrive/Katalog/model.pkl")

# Mapování mezi interními názvy tříd a požadovanými názvy adresářů
class_mapping = {
    'tensor(0)': 'earring',
    'tensor(1)': 'chain',
    'tensor(2)': 'childrensEarring',
    'tensor(3)': 'mensRing',
    'tensor(4)': 'necklace',
    'tensor(5)': 'pendant',
    'tensor(6)': 'ring',
    'tensor(7)': 'weddingRing'
}

# Vytvoření instance ImageClassifier a zpracování obrázků
image_classifier = ImageClassifier(model_dir, classified_dir, class_mapping)
image_classifier.process_images(unclassified_dir)

Třída otestuje nastavený obrázek na podobnost v nastavené kategorii a ukáže cestu k němu.

In [ ]:
##Třída testuje podobnost s obrázkem v nastavené kategorii
class ImageSimilarityClassifier:
    def __init__(self, model_path, test_image_path, similar_images_path, threshold=0.95):
        # Inicializace třídy s cestami k modelu, testovacímu obrázku a složce s podobnými obrázky
        self.model_path = Path(model_path)
        self.test_image_path = Path(test_image_path)
        self.similar_images_path = Path(similar_images_path)
        self.threshold = threshold

        # Definice DataLoaderů pro podobné obrázky
        self.dls = DataBlock(
            blocks=(ImageBlock, CategoryBlock),
            get_items=get_image_files,
            splitter=RandomSplitter(valid_pct=0.2, seed=42),
            get_y=parent_label,
            item_tfms=[Resize(192, method='squish')]
        ).dataloaders(self.similar_images_path, bs=30)

        # Načtení modelu pro predikci
        self.learn = load_learner(self.model_path)

        # Vybrání konkrétní vrstvy modelu pro extrakci rysů
        self.target_layer = self.learn.model[0][-2][0]

        # Registrování hook funkce na vybrané vrstvě
        self.hook = self.target_layer.register_forward_hook(self.hook_fn)
        self.hook_output = None

    def hook_fn(self, m, i, o):
        # Hook funkce pro extrakci výstupu z vrstvy
        self.hook_output = o

    def load_test_image(self):
        # Načtení testovacího obrázku
        return PILImage.create(self.test_image_path)

    def extract_feature_vector(self, img):
        # Extrakce vektoru rysů z obrázku
        with torch.no_grad():
            self.learn.predict(img)
        return self.hook_output[0].view(-1)

    def calculate_similarity(self, vector1, vector2):
        # Výpočet kosínové podobnosti mezi dvěma vektory
        return 1 - cosine(vector1, vector2)

    def find_similar_images(self):
        # Hledání podobných obrázků
        feature_vector = self.extract_feature_vector(self.load_test_image())
        similar_images = []

        for img_path in get_image_files(self.similar_images_path):
            img = PILImage.create(img_path)
            ring_feature_vector = self.extract_feature_vector(img)
            similarity = self.calculate_similarity(feature_vector, ring_feature_vector) #hodnota podobnosti
            if similarity >= self.threshold:
                print("Cesta k hledanému obrázku -",img_path)
                similar_images.append((img, similarity))

        similar_images.sort(key=lambda x: x[1], reverse=True)
        # Získání názvu obrázku z cesty souboru
        #similar_image_name = os.path.basename(similar_images[0][0].filename)
        #print(similar_image_name)
        return similar_images

    def display_similar_images(self, num_similar=5):
        # Zobrazení podobných obrázků s názvy souborů
        similar_images = self.find_similar_images()
        num_similar = min(num_similar, len(similar_images))
        fig, axes = plt.subplots(1, num_similar, figsize=(15, 5))


        for i in range(num_similar):
            img, similarity = similar_images[i]
            ax = axes[i]
            #img_path = str(img)  # Převod Path objektu na řetězec
            #img_name = os.path.basename(img_path)  # Získání názvu souboru
            #img.show(ax=ax, title=f"Name: {img_name}\nSimilarity: {similarity:.2f}")
            img.show(ax=ax, title=f"Similarity: {similarity:.2f}")
            ax.axis('off')

        plt.show()

test_image_path = "/content/drive/MyDrive/KatalogAA/Sperky/obr_1.jpg"
model_dir = Path("/content/drive/MyDrive/Katalog/model.pkl")
similar_images_path = "/content/drive/MyDrive/Katalog/sperk/ring"
threshold = 0.95

# Vytvoření instance třídy
image_similarity_classifier = ImageSimilarityClassifier(model_dir, test_image_path, similar_images_path, threshold)

# Zobrazení podobných obrázků s názvy souborů
image_similarity_classifier.display_similar_images()

Úprava obrázků - přejmenování a úprava velikosti.

In [ ]:
import os
from google.colab.patches import cv2_imshow
import cv2



# Třída pro práci s obrázky
class ImageProcessor:
    # Konstruktor třídy
    def __init__(self, base_path, categories, new_size):
        self.base_path = base_path # Základní cesta k adresáři s obrázky
        self.categories = categories # Seznam názvů adresářů
        self.new_size = new_size # Nová velikost obrázků

    # Metoda pro přejmenování obrázků
    def rename(self):
        # Procházení adresářů
        for category in self.categories:
            category_path = os.path.join(self.base_path, category)
            if os.path.isdir(category_path):
                # Procházení souborů v adresáři
                for idx, filename in enumerate(os.listdir(category_path)):
                    if filename.endswith(".jpg"):  # Předpokládáme, že jsou to obrázky ve formátu JPG
                        new_name = f"{category}_{idx + 1}.jpg"
                        old_path = os.path.join(category_path, filename)
                        new_path = os.path.join(category_path, new_name)
                        os.rename(old_path, new_path)
                        print(f"Přejmenován soubor: {filename} na {new_name}")
            else:
                print(f"Adresář {category_path} neexistuje.")

    # Metoda pro změnu velikosti obrázků
    def resize(self):
        # Procházení adresářů
        for category in self.categories:
            category_path = os.path.join(self.base_path, category)
            if os.path.isdir(category_path):
                # Procházení souborů v adresáři
                for filename in os.listdir(category_path):
                    if filename.endswith(".jpg"):  # Předpokládáme, že jsou to obrázky ve formátu JPG
                        file_path = os.path.join(category_path, filename)
                        image = cv2.imread(file_path) # Načtení obrázku
                        resized_image = cv2.resize(image, self.new_size) # Změna velikosti obrázku
                        cv2.imwrite(file_path, resized_image) # Uložení nového obrázku na stejnou cestu
                        print(f"Změněna velikost souboru: {filename}")
            else:
                print(f"Adresář {category_path} neexistuje.")

# Cesta k adresáři obsahující obrázky
base_path = '/content/drive/MyDrive/Katalog/sperk'

# Seznam názvů adresářů (doplnit seznam, které kategorie upraviji, 'pendant', 'earring' atd.)
categories = [
    "ring"
]


# Vytvoření instance třídy ImageProcessor s novou velikostí 300x300 pixelů
ip = ImageProcessor(base_path, categories, (300, 300))

# Volání metody rename pro přejmenování obrázků
ip.rename()

# Volání metody resize pro změnu velikosti obrázků
ip.resize()


Vybraný obrázek ořízne podle hlavního (největšího) objektu na obrázku (funguje dobře pro prsten)

In [ ]:
import cv2 # Potřebné pro načtení, zpracování a ukládání obrázků
from google.colab.patches import cv2_imshow # Potřebné pro zobrazení obrázků v Google Colab
from google.colab import output # Potřebné pro registraci zpětného volání
from IPython.display import Javascript # Potřebné pro zobrazení dialogového okna
from google.colab import files # Potřebné pro nahrávání souborů
import time # Potřebné pro nastavení prodlevy

# Třída pro ořezávání obrázků podle největší kontury
class ImageCropper:
    # Konstruktor třídy
    def __init__(self, base_path):
        self.base_path = base_path # Základní cesta k adresáři s obrázky

    # Metoda pro načtení obrázku
    def load_image(self):
        # Nahrajte obrázek ze svého Google Disku
        uploaded = files.upload()
        image_filename = list(uploaded.keys())[0]

        # Odstranění pořadového čísla z názvu souboru
        image_name = image_filename.split(' (')[0]
        # Zobrazení názvu vybraného souboru
        print(f"Název vybraného souboru: {image_name}")

        # Načtení obrázku
        img = cv2.imread(image_filename)
        #img = cv2.imread('/content/drive/MyDrive/KatalogAA/obr.jpg')

        return img, image_name

    # Metoda pro ořezání obrázku podle největší kontury
    def crop_image(self, img):
        # Převod obrázku na odstíny šedé
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Použití detektoru hran Canny k nalezení hran na obrázku
        edges = cv2.Canny(gray, 30, 100)

        # Nalezení kontur v obrázku hran
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Nalezení kontury s největší plochou
        c = max(contours, key=cv2.contourArea)

        # Nakreslení ohraničujícího rámečku kolem kontury
        x, y, w, h = cv2.boundingRect(c)
        paddingX = 110 # Přidání nějakého odsazení pro vytvoření většího ohraničujícího rámečku
        paddingY = 120
        cv2.rectangle(img, (x - paddingX, y - paddingY), (x + w + paddingX, y + h + paddingY), (0, 255, 0), 3)

        # Oříznutí obrázku podle ohraničujícího rámečku
        cropped_img = img[y - paddingY:y + h + paddingY, x - paddingX:x + w + paddingX]

        return cropped_img

    # Metoda pro zobrazení oříznutého obrázku a vyzvání k uložení
    def show_and_save(self, cropped_img, image_name):
        # Zobrazení oříznutého obrázku
        cv2_imshow(cropped_img)
        delay = 6  # Nastavení prodlevy
        time.sleep(delay)

        # Získejte rozměry obrázku
        height, width, _ = cropped_img.shape

        # Ořízněte obrázek o 4 pixely na okrajích
        cropped_img = cropped_img[4:height-4, 4:width-4]

        # Vyzvání k potvrzení nebo zamítnutí uložení oříznutého obrázku
        def save_image():
            cropped_image_name = self.base_path+'cropped_' + image_name +".jpg"
            print("Název a cesta souboru:",cropped_image_name)
            cv2.imwrite(cropped_image_name, cropped_img)
            print('Oříznutý obrázek byl uložen.')

        def on_confirm(confirm):
            if confirm:
                save_image()
            else:
                print('Oříznutý obrázek nebyl uložen.')

        print('Chcete uložit oříznutý obrázek?')
        output.register_callback('on_confirm', on_confirm)
        display(Javascript('google.colab.kernel.invokeFunction("on_confirm", [confirm("Chcete uložit oříznutý obrázek?")], {})'))

# Vytvoření instance třídy ImageCropper s cestou k adresáři s obrázky
ic = ImageCropper('/content/drive/MyDrive/KatalogAA/')

# Volání metody load_image pro načtení obrázku
img, image_name = ic.load_image()

# Volání metody crop_image pro ořezání obrázku podle největší kontury
cropped_img = ic.crop_image(img)

# Volání metody show_and_save pro zobrazení oříznutého obrázku a vyzvání k uložení
ic.show_and_save(cropped_img, image_name)
